<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/dbdevrel/blob/main/source-codes/colab-code/dbconnect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install oracledb

In [ ]:
import os
import oracledb
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Connection parameters
#connection_string = "indeducation_high"  # Update with your TNS alias from tnsnames.ora
username = "DEMOUSER"  # Update with your username
password = "Welcome123456#"  # Update with your password
tns_name = "indeducation_high"
wall_config_dir = "/content/drive/MyDrive/Wallet_IndEducation"
wall_pwd = "welcome1"
table = "MYNOTES"
col = "NOTES"
try:
    # Connect to Oracle Database
    conn = oracledb.connect(user=username,
                              password=password,
                              dsn=tns_name,
                              config_dir=wall_config_dir,
                              wallet_location=wall_config_dir,
                              wallet_password=wall_pwd)

    print("✓ Successfully connected to Oracle Database")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    connection = None

In [ ]:
# Query mynotes table
if conn:
    try:
        cursor = conn.cursor()

        # Execute query
        cursor.execute("""
                          create table shops (
                          shop_id number constraint pk_shop_id primary key,
                          shop_name varchar2(40),
                          location varchar2(40)
                          )
                      """)
        cursor.execute("""
                          create table shop_items (
                          item_no number  constraint pk_item_no primary key,
                          item_name varchar2(40),
                          quantity_available number,
                          unit_price number,
                          shop_id number constraint fk_shop_id references shops
                      )
                      """)
        conn.commit()

        print("Shops and Shop_Items table created")
        cursor.close()
    except Exception as e:
        print(f"✗ Query failed: {e}")
else:
    print("✗ No database connection available")

In [ ]:
# Query mynotes table
if conn:
    try:
        cursor = conn.cursor()


        # Execute query
        cursor.execute("""
            CREATE JSON RELATIONAL DUALITY VIEW shop_duality_view AS
                shops @insert @update @delete
                {
                    _id               : shop_id,
                    shop_name         : shop_name,
                    location          : location,
                    items             : shop_items @insert @update @delete
                    [{
                        item_no            : item_no,
                        item_name          : item_name,
                        quantity_available : quantity_available,
                        unit_price         : unit_price
                    }]
                }
        """)

        conn.commit()

        print("Shops_DV created  ")
        cursor.close()
    except Exception as e:
        print(f"✗ Query failed: {e}")
else:
    print("✗ No database connection available")

In [ ]:
if conn:
    try:
        cur = conn.cursor()
        # -------------------------------------------------
        # 1️⃣  Insert two shops with items
        # -------------------------------------------------
        cur.execute("""
            INSERT INTO shop_duality_view (data) VALUES (
                '{"_id":100,
                  "shop_name":"Tech Haven",
                  "location":"New York",
                  "items":[{"item_no":101,
                            "item_name":"Laptop",
                            "quantity_available":15,
                            "unit_price":1299.99},
                           {"item_no":102,
                            "item_name":"Wireless Mouse",
                            "quantity_available":42,
                            "unit_price":25.50}]}
            ')
        """)

        cur.execute("""
            INSERT INTO shop_duality_view (data) VALUES (
                '{"_id":200,
                  "shop_name":"Grocery Corner",
                  "location":"Los Angeles",
                  "items":[{"item_no":201,
                            "item_name":"Organic Apples",
                            "quantity_available":80,
                            "unit_price":1.20},
                           {"item_no":202,
                            "item_name":"Whole‑Grain Bread",
                            "quantity_available":30,
                            "unit_price":2.75}]}
            ')
        """)

        conn.commit()
        print("✅ Sample shops and items inserted via the duality view.")
        cur.close()
    except Exception as e:
        print(f"✗ Insert failed: {e}")
else:
    print("✗ No database connection available")

In [ ]:
if conn:
  try:
        cur = conn.cursor()

        result = cursor.execute("""
            SELECT JSON_SERIALIZE(data pretty) AS shop_json
            FROM   shop_duality_view
        """)
        for row in result:
            print(row[0])
  except Exception as e:
        print(f"✗ SELECT failed: {e}")
else:
    print("✗ No database connection available")

In [ ]:

# Close database connection
if conn:
    try:
        conn.close()
        print("✓ Database connection closed")
    except Exception as e:
        print(f"✗ Error closing connection: {e}")